Drawing a subset of data from biendata

In [1]:
import os, shutil, csv

In [2]:
original_dataset_dir = 'D:\Wastesorting\Train\images_withoutrect' #directory name of your biendata
base_dir = './data_small' #create a directory for the data subset

os.mkdir(base_dir)

#creating a new folder for each set
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation') 
os.mkdir(validation_dir) 
test_dir = os.path.join(base_dir, 'test') 
os.mkdir(test_dir)

In [16]:
#making a folder for each category inside train folder
#there are 204 categories with ID 1..204

for i in range(1,205):
    train_categories_dir = os.path.join(train_dir, str(i))
    os.mkdir(train_categories_dir)
    
for i in range(1,205):
    validation_categories_dir = os.path.join(validation_dir, str(i))
    os.mkdir(validation_categories_dir)

In [6]:
#importing csv files, and change it into a list

with open('train.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

In [8]:
data.pop(0)

['category_id', 'image_id', 'file_name']

In [9]:
data

[['1', '11505', 'images_withoutrect/11505.png'],
 ['1', '11506', 'images_withoutrect/11506.png'],
 ['1', '11507', 'images_withoutrect/11507.png'],
 ['1', '11508', 'images_withoutrect/11508.png'],
 ['1', '11509', 'images_withoutrect/11509.png'],
 ['1', '11510', 'images_withoutrect/11510.png'],
 ['1', '11511', 'images_withoutrect/11511.png'],
 ['1', '11513', 'images_withoutrect/11513.png'],
 ['1', '11514', 'images_withoutrect/11514.png'],
 ['1', '11515', 'images_withoutrect/11515.png'],
 ['1', '11516', 'images_withoutrect/11516.png'],
 ['1', '11518', 'images_withoutrect/11518.png'],
 ['1', '11519', 'images_withoutrect/11519.png'],
 ['1', '11521', 'images_withoutrect/11521.png'],
 ['1', '11523', 'images_withoutrect/11523.png'],
 ['1', '11524', 'images_withoutrect/11524.png'],
 ['1', '11527', 'images_withoutrect/11527.png'],
 ['1', '11529', 'images_withoutrect/11529.png'],
 ['1', '11530', 'images_withoutrect/11530.png'],
 ['1', '11531', 'images_withoutrect/11531.png'],
 ['1', '11532', 'ima

In [ ]:
train_copied=0 # training images copied for each category
validation_copied=0 # validation images copied for each category
last_cat=0 # records category of the last copied image
active=True # defines if we need to copy images or not in each loop
for entry in data:
    if active:
        if train_copied<10:
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(train_dir, entry[0], entry[1]+'.png'))
            train_copied+=1
        elif validation_copied<3:
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(validation_dir, entry[0], entry[1]+'.png'))
            validation_copied+=1
        else:
            active=False
        last_cat=entry[0]
    else:
        if entry[0] != last_cat:
            train_copied=0
            validation_copied=0
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(train_dir, entry[0], entry[1]+'.png'))
            train_copied+=1
            active=True

In [ ]:
#checking if it is indeed the amount
print('total training images:', len(os.listdir(train_cats_dir)))

Training model building

In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine import Input, Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
import keras

Using TensorFlow backend.


As of keras 2, the module keras.layers.merge doesn't have a generic public Merge-Layer. Instead you are supposed to import the subclasses like keras.layers.Add or keras.layers.Concatenate etc. directly (or their functional interfaces with the same names lowercase: keras.layers.add, keras.layers.concatenate etc.).

In [2]:
#complete building function of Resnet x Inception v2
# we reduce # filters by factor of 8 compared to original inception-v4
nb_filters_reduction_factor = 8

def inception_resnet_v2_stem(x):
    # in original inception-resnet-v2, conv stride is 2
    x = Convolution2D(32//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    x = Convolution2D(32//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    x = Convolution2D(64//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    # in original inception-resnet-v2, stride is 2
    a = MaxPooling2D((3, 3), strides=(1, 1), border_mode='valid', dim_ordering='tf')(x)
    # in original inception-resnet-v2, conv stride is 2
    b = Convolution2D(96//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(x)
   # x = merge([a, b], mode='concat', concat_axis=-1)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    a = Convolution2D(64//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    a = Convolution2D(96//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(a)
    b = Convolution2D(64//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Convolution2D(64//nb_filters_reduction_factor, 7, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Convolution2D(64//nb_filters_reduction_factor, 1, 7, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Convolution2D(96//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(b)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    # in original inception-resnet-v2, conv stride should be 2
    a = Convolution2D(192//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    # in original inception-resnet-v2, stride is 2
    b = MaxPooling2D((3, 3), strides=(1, 1), border_mode='valid', dim_ordering='tf')(x)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    x = Activation('relu')(x)
    return x


def inception_resnet_v2_A(x):
    shortcut = x
    
    a = Convolution2D(32//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    b = Convolution2D(32//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Convolution2D(32//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    
    c = Convolution2D(32//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Convolution2D(48//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(c)
    c = Convolution2D(64//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    x = Convolution2D(384//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x
def inception_resnet_v2_reduction_A(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid', dim_ordering='tf')(x)
    b = Convolution2D(384//nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    c = Convolution2D(256//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Convolution2D(256//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(c)
    c = Convolution2D(384//nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    
    return x
    

def inception_resnet_v2_B(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    b = Convolution2D(128//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Convolution2D(160//nb_filters_reduction_factor, 1, 7, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Convolution2D(192//nb_filters_reduction_factor, 7, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(1154//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

def inception_resnet_v2_reduction_B(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid', dim_ordering='tf')(x)
    b = Convolution2D(256//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Convolution2D(288//nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(b)
    c = Convolution2D(256//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Convolution2D(288//nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(c)
    d = Convolution2D(256//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    d = Convolution2D(288//nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(d)
    d = Convolution2D(320//nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
                      init='he_normal', border_mode='valid', dim_ordering='tf')(d)
    
    x = keras.layers.concatenate([a,b,c,d],axis=-1)
    
    return x


def inception_resnet_v2_C(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
    b = Convolution2D(192//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Convolution2D(224//nb_filters_reduction_factor, 1, 3, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Convolution2D(256//nb_filters_reduction_factor, 3, 1, subsample=(1, 1), activation='relu',
                      init='he_normal', border_mode='same', dim_ordering='tf')(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(2048//nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
                      init='he_normal', border_mode='same', dim_ordering='tf')(x)
    
   # x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

In [4]:
img_rows, img_cols = 1920, 1080
img_channels = 3

nb_classes = 198 

# in original inception-resnet-v2, these are 5, 10, 5, respectively
num_A_blocks = 1
num_B_blocks = 1
num_C_blocks = 1

inputs = Input(shape=(img_rows, img_cols, img_channels))

x = inception_resnet_v2_stem(inputs)
for i in range(num_A_blocks):
    x = inception_resnet_v2_A(x)
x = inception_resnet_v2_reduction_A(x)
for i in range(num_B_blocks):
    x = inception_resnet_v2_B(x)
x = inception_resnet_v2_reduction_B(x)
for i in range(num_C_blocks):
    x = inception_resnet_v2_C(x)

x = AveragePooling2D(pool_size=(4, 4), strides=(1, 1), border_mode='valid', dim_ordering='tf')(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", strides=(1, 1), padding="valid", data_format="channels_last", kernel_initializer="he_normal")`
  
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", strides=(1, 1), padding="valid", data_format="channels_last", kernel_initializer="he_normal")`
  # Remove the CWD from sys.path while we load stuff.
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", strides=(1, 1), padding="same", data_format="channels_last", kernel_initializer="he_normal")`
  if sys.path[0] == '':
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: 

C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:128: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu", strides=(2, 2), padding="valid", data_format="channels_last", kernel_initializer="he_normal")`
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:139: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (1, 1), activation="relu", strides=(1, 1), padding="same", data_format="channels_last", kernel_initializer="he_normal")`
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:142: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (1, 1), activation="relu", strides=(1, 1), padding="same", data_format="channels_last", kernel_initializer="he_normal")`
C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:144: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(28, (1, 3), activation="relu", strides=(1, 1), padding="same", data_form

ResourceExhaustedError: OOM when allocating tensor with shape[31846144,198] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1920, 1080, 3 0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 1918, 1078, 4 112         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 1916, 1076, 4 148         conv2d_52[0][0]                  
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 1916, 1076, 8 296         conv2d_53[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [16]:
model.compile(optimizer='adam',  #optimizer=optimizers.RMSprop(lr=2e-5), could also try that
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

In [17]:
#setting up all the parameter for training
batch_size = 64 #try 32, 128
epoch = 50 # try 50, 100.
data_augmentation = True

# preprocessing the data

1 Read the picture files.

2 Decode the JPEG content to RGB grids of pixels.

3 Convert these into floating-point tensors.

4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values).

In [ ]:
#preprocessing the data implementation
#using data implementation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.4,
horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #train_dir is the path where you store all the validaiton folder, chnage this
    target_size = (1980,1920),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir, #train_dir is the path where you store all the validation folder, change this
    target_size = (1980,1920),
    batch_size = batch_size,
    class_mode = 'categorical')



In [ ]:
#training the model, fit the data
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=epoch,
validation_data=validation_generator,
validation_steps=50)

In [ ]:
model.save('waste_sort_with_resnet_inception_v2.h5')